In [52]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import load_model
from keras.models import save_model

In [53]:
def get_nb_files(directory):
  """Get number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt


In [54]:
def setup_to_transfer_learn(model, base_model):
  """Freeze all layers and compile the model"""
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [55]:
def add_new_last_layer(base_model, nb_classes):
  """Add last layer to the convnet
  Args:
    base_model: keras model excluding top
    nb_classes: # of classes
  Returns:
    new keras model with last layer
  """
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
  predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
  model = Model(input=base_model.input, output=predictions)
  return model


In [56]:
def plot_training(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))

  plt.plot(epochs, acc, 'r.')
  plt.plot(epochs, val_acc, 'r')
  plt.title('Training and validation accuracy')

  plt.figure()
  plt.plot(epochs, loss, 'r.')
  plt.plot(epochs, val_loss, 'r-')
  plt.title('Training and validation loss')
  plt.show()


In [81]:
"""Use transfer learning and fine-tuning to train a network on a new dataset"""

#setup datasets and variables

train_data_dir = 'D:\\Melanoma Class\\HAM datasets\\datasets\\train_dir'
test_data_dir = 'D:\\Melanoma Class\\HAM datasets\\datasets\\val_dir'


nb_train_samples = get_nb_files('D:\\Melanoma Class\\HAM datasets\\datasets\\train_dir')
nb_classes = len(glob.glob('D:\\Melanoma Class\\HAM datasets\\datasets\\train_dir' + "/*"))
nb_val_samples = get_nb_files('D:\\Melanoma Class\\HAM datasets\\datasets\\val_dir')
nb_epoch = 3
batch_size = 32
    
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 3
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 300

In [82]:
nb_classes

2

In [84]:
#initialize InceptionV3 model without top layer

base_model = InceptionV3(include_top=False, weights='imagenet', input_shape = (IM_WIDTH, IM_HEIGHT, 3)) 



KeyboardInterrupt: 

In [ ]:
#freeze layers

for layer in base_model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
    layer.trainable = False
    

In [ ]:
#add top layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
#compile model

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [78]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, 149, 149, 32) 96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, 149, 149, 32) 0           batch_normalization_283[0][0]    
__________________________________________________________________________________________________
conv2d_284

__________________________________________________________________________________________________
average_pooling2d_36 (AveragePo (None, 8, 8, 2048)   0           mixed9[0][0]                     
__________________________________________________________________________________________________
conv2d_368 (Conv2D)             (None, 8, 8, 320)    655360      mixed9[0][0]                     
__________________________________________________________________________________________________
batch_normalization_370 (BatchN (None, 8, 8, 384)    1152        conv2d_370[0][0]                 
__________________________________________________________________________________________________
batch_normalization_371 (BatchN (None, 8, 8, 384)    1152        conv2d_371[0][0]                 
__________________________________________________________________________________________________
batch_normalization_374 (BatchN (None, 8, 8, 384)    1152        conv2d_374[0][0]                 
__________

In [64]:
#initialize test and training sets

train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    fill_mode="nearest",
    
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
  )

test_datagen = ImageDataGenerator(
    rescale=1./255,
    fill_mode="nearest",
    
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
  )

train_generator = train_datagen.flow_from_directory(
    'D:\\Melanoma Class\\HAM datasets\\datasets\\train_dir',
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
    class_mode='binary'
  )

validation_generator = test_datagen.flow_from_directory(
    'D:\\Melanoma Class\\HAM datasets\\datasets\\val_dir',
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
    class_mode='binary'
  )

Found 9015 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [65]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

In [66]:
file_path = '180501_best_weights.h5'

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1 )




In [71]:
history_tl = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples/batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=nb_val_samples/batch_size,
    class_weight='auto',
    callbacks = [checkpoint]
)


Epoch 1/5


ValueError: Error when checking target: expected dense_6 to have shape (2,) but got array with shape (1,)

In [ ]:
#plot


In [ ]:
import h5py

model.save('180501_tl1.h5')

In [1]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('D:\\Melanoma Class\\HAM datasets\\datasets\\val_dir\\malignant\\ISIC_0033569.jpg', 
                            target_size = (IM_WIDTH, IM_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

result

C:\Users\Dennis\Anaconda3\envs\tensorflow_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


NameError: name 'IM_WIDTH' is not defined

In [46]:
test_image = image.load_img('D:\\Melanoma Class\\HAM datasets\\datasets\\val_dir\\benign\\ISIC_0033409.jpg', 
                            target_size = (IM_WIDTH, IM_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

result

array([[1., 0.]], dtype=float32)